In [88]:
# -*- coding:utf-8 -*-
import urllib
import urllib2
import re
import pandas as pd
def getPage(curr_url):
    try:
        request = urllib2.Request(curr_url)
        response = urllib2.urlopen(request)
        return response.read()
    except urllib2.URLError, e:
        if hasattr(e,"reason"):
            print u'cannot connect to cars',e.reason
            return None
def get_cars(curr_page):
    car_list_temp = []
    pattern = re.compile("data-goto-vdp=.(.*?). data.*?"
                         "standard-link=.md-ymmt.>\n                        (.*?)\n                    </a>.*?</sup>(.*?)\n.*?</span>.*?row__mileage..(.*?) mi"+\
                         ".*?<div class=.listing-row__meta-item.>(.*?)</div>"+\
                         ".*?<div class=.listing-row__meta-item.>(.*?)</div>"+\
                         ".*?phone-control.>\n(.*?)\n                            </span>", re.S)
    result = re.findall(pattern, curr_page)
    #print result
    for i in range(0, len(result)):
        new_car = car(result[i][0], result[i][1], result[i][2], result[i][3], result[i][4], result[i][5], result[i][6])
        new_car.dealer_location(curr_page)
        car_list_temp.append(new_car)
    return car_list_temp
class car:
    def __init__(self, ID, name, price, miles, looking, powertrain, phone):
        self.id = int(ID)
        self.name = name
        self.price = int(price.replace(",", ""))
        self.miles = int(miles.replace(",", ""))
        self.looking = looking
        self.powertrain = powertrain
        temp = phone.replace(" ", "")
        temp = "\\"+temp[:5] +" "+temp[5:]
        self.phone = temp[:5] +"\\"+temp[5:]
    def show(self):
        print "id:", self.id
        print "name:", self.name
        print "price:", self.price
        print "miles:", self.miles
        print "looking:", self.looking
        print "powertrain:", self.powertrain
        print "phone:", self.phone
        print "state:", self.state
    def dealer_location(self, page):
        find_string = "telephone.:." + self.phone+".*?addressRegion.:.(.*?).,"
        location_pattern = re.compile(find_string, re.S)
        result = re.findall(location_pattern, page)
        self.state = result[0]
    def to_dict(self):
        curr_dict = {
            'id' : self.id,
            'name' : self.name,
            'price' : self.price,
            'miles' : self.miles,
            'looking' : self.looking,
            'powertrain' : self.powertrain,
            'state' : self.state
        }
        return curr_dict
class car_set:
    def __init__(self, baseUrl, pageNum):
        self.car_list = []
        self.baseURL = baseUrl
        self.pageNum = int(pageNum)
        self.basePage = getPage(self.baseURL)
        self.page_list = self.setPageList()
        
        for j in range(0, len(self.page_list)):
            print j
            if j==36 or j==38:
                continue
            curr_page = getPage(self.page_list[j])
            self.car_list = self.car_list + get_cars(curr_page)
    def setPageList(self):
        self.page_list = []
        self.page_list.append(self.baseURL)
        for i in range(2, self.pageNum+1):
            new_page = self.baseURL.replace("page=1", "page="+str(i))
            self.page_list.append(new_page)
        return self.page_list
    def dic_list(self):
        dict_list=[]
        for i in range(0, len(self.car_list)):
            dict_list.append(self.car_list[i].to_dict())

In [89]:
baseURL = raw_input("input the url: ")
pageNum = raw_input("how many pages? ")
my_set = car_set(baseURL, pageNum)
dict_list=[]
for i in range(0, len(my_set.car_list)):
    dict_list.append(my_set.car_list[i].to_dict())
acura_df = pd.DataFrame(dict_list)

input the url: https://www.cars.com/for-sale/searchresults.action/?mdId=21906&mkId=20028&page=1&perPage=100&rd=99999&searchSource=GN_REFINEMENT&sf1Dir=DESC&sf1Nm=price&slrTypeId=28878&stkTypId=28881&zc=60606
how many pages? 39
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38


In [90]:
print len(acura_df)
print len(acura_df.id.unique())

3536
3536


In [91]:
acura_df.head()

,id,looking,miles,name,powertrain,price,state
0,699339412,Iridium Silver Metallic (ext) | Black (int),173,Used 2015 Mercedes-Benz S65 AMG,7-Speed Automatic | RWD,193500,CA
1,698280411,Black (ext) | Designo Porcelain (int),463,Used 2015 Mercedes-Benz S65 AMG,7-Speed Automatic | RWD,179888,CA
2,697662693,Iridium Silver (ext) | Designo Black Exclusive...,9149,Certified 2015 Mercedes-Benz S65C,7-SPEED AUTOMATIC | RWD,169872,VA
3,696463973,Designo Diamond White Metallic (ext) | Designo...,2067,Used 2015 Mercedes-Benz S65 AMG,7-Speed Automatic | RWD,166900,NY
4,697860102,Designo Magno Alanite Gray (ext) | Black (int),7614,Used 2015 Mercedes-Benz S65 AMG,7-Speed Automatic | RWD,165500,GA


In [92]:
acura_df.to_csv("./dataset/Benz_S.csv")